# Deploy ADK Agent to Vertex AI Agent Engine

**references**

* [code src](https://github.com/googleapis/python-aiplatform/blob/main/vertexai/agent_engines/_agent_engines.py)
* (Vertex docs) [deploy an agent](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/deploy)
* (ADK docs) [deploy ADK to Agent Engine](https://google.github.io/adk-docs/deploy/agent-engine/#step-5-deploy-to-agent-engine)
* [testing deployed ADK agent](https://google.github.io/adk-docs/get-started/testing/#deploying-your-agent)

## setup

Note: after upgrading to `google-cloud-aiplatform==1.112.0` | `vertexai==1.112.0` see [agent engine migration](https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/agent-engine-migration#before) docs

In [1]:
import os, sys
import dotenv

import vertexai
vertexai.__version__

'1.112.0'

Get the current working directory...

In [2]:
os.getcwd()

'/usr/local/google/home/jordantotten/adk_pipe'

In [3]:
# don't forget we have an `.env` file under trend_trawler dir
!tree -I "creative_agent|stash|imgs"

.
├── deploy-to-agent-engine.ipynb
├── poetry.lock
├── pyproject.toml
├── README.md
└── trend_trawler
    ├── agent.py
    ├── callbacks.py
    ├── config.py
    ├── __init__.py
    ├── requirements.txt
    └── tools.py

2 directories, 10 files


In [4]:
sys.version, sys.executable

('3.13.5 (main, Jun 25 2025, 18:55:22) [GCC 14.2.0]',
 '/usr/local/google/home/jordantotten/adk_pipe/.venv/bin/python')

# Deploy `trend_trawler` agent

* test locally 
* deploy to Agent Engine
* test deployment

## config

In [5]:
# Load environment variables set with agent (./trend_trawler/.env)
dotenv.load_dotenv(dotenv_path = os.path.join('trend_trawler', '.env'))

PROJECT_ID=os.environ.get("GOOGLE_CLOUD_PROJECT")
PROJECT_NUMBER=os.environ.get("GOOGLE_CLOUD_PROJECT_NUMBER")
GCS_BUCKET_NAME=os.environ.get("GOOGLE_CLOUD_STORAGE_BUCKET")
LOCATION=os.environ.get("GOOGLE_CLOUD_LOCATION")

# Vertex AI Reasoning Engine (default) service agent
SERVICE_ACCOUNT = f"service-{PROJECT_NUMBER}@gcp-sa-aiplatform-re.iam.gserviceaccount.com"

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUMBER: {PROJECT_NUMBER}")
print(f"GCS_BUCKET_NAME: {GCS_BUCKET_NAME}")
print(f"LOCATION: {LOCATION}")
print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

PROJECT_ID: hybrid-vertex
PROJECT_NUMBER: 934903580331
GCS_BUCKET_NAME: trend-trawler-deploy-ae
LOCATION: us-central1
SERVICE_ACCOUNT: service-934903580331@gcp-sa-aiplatform-re.iam.gserviceaccount.com


In [ ]:
!gcloud projects get-iam-policy {PROJECT_NUMBER} --flatten="bindings" --format="table(bindings.role)" --filter="bindings.members:'{SERVICE_ACCOUNT}'"

ROLE
roles/aiplatform.reasoningEngineServiceAgent
roles/aiplatform.user
roles/bigquery.admin
roles/cloudsql.client
roles/cloudsql.instanceUser
roles/editor
roles/serviceusage.serviceUsageConsumer
roles/storage.admin
roles/visionai.admin


Create GCS bucket (if needed)...

In [11]:
!gcloud storage buckets create gs://$GCS_BUCKET_NAME --location=$LOCATION

Creating gs://trend-trawler-deploy-ae/...


In [6]:
# Vertex AI Client Initialization:
client = vertexai.Client(
    project=PROJECT_ID, 
    location=LOCATION,
) # pyright: ignore[reportCallIssue]


## Test locally

> Before deploying ADK agent to Vertex AI Agent Engine, test locally with the SDK.

An ADK application

> [src code](https://github.com/googleapis/python-aiplatform/blob/main/vertexai/agent_engines/templates/adk.py#L318)

```python
from vertexai.agent_engines import AdkApp

app = AdkApp(
    agent=root_agent,
    enable_tracing=True,

    # Defaults to a callable that returns InMemorySessionService
    session_service_builder=None,

    # Defaults to a callable that returns InMemoryArtifactService
    artifact_service_builder=None,
    
    # Defaults to a callable that returns InMemoryMemoryService when running locally 
    # and VertexAiMemoryBankService when running on Agent Engine.
    memory_service_builder=None,

    # If not provided, a default instrumentor builder will be used
    instrumentor_builder=None,
)
```

In [11]:
from trend_trawler.agent import root_agent

root_agent

/usr/local/google/home/jordantotten/adk_pipe/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
INFO:root:root_dir: /usr/local/google/home/jordantotten/adk_pipe


LlmAgent(name='root_agent', description='Determines culturally relevant Search trends to use for ad creatives.', parent_agent=None, sub_agents=[], before_agent_callback=[<function _load_session_state at 0x7f8d3f1b1940>], after_agent_callback=None, model='gemini-2.5-flash', instruction="You are an Expert AI Marketing Research & Strategy Assistant.\n\n    Once initiated, immediately begin the steps below.\n\n    1. Call `gather_trends_agent` as a tool to gather the latest Google Search Trends. \n    2. Call `understand_trends_agent` as a tool to conduct web research about each trending topic.\n    3. Call `pick_trends_agent` as a tool to determine the most relevant subset of trends for this campaign. \n    4. For each trending topic in the 'selected_gtrends' state key, call the `save_search_trends_to_session_state` tool to save them to the session state.\n\n    Once these three tasks are complete, complete the following actions:\n\n    Action 1: Save to markdown File\n    Call the `write

To make your agent compatible with Agent Engine, you need to wrap it in an AdkApp object.

In [ ]:
from vertexai.agent_engines import AdkApp

app = AdkApp(
    agent=root_agent,
    app_name="local-trend-trawler",
    enable_tracing=True,
)

app

In [ ]:
session = await app.async_create_session(user_id="u_123")
session

Session(id='ff5c468d-813d-4299-b767-b1ef576103c1', app_name='local-trend-trawler', user_id='u_123', state={}, events=[], last_update_time=1757635751.549083)

In [ ]:
await app.async_list_sessions(user_id="u_123")

ListSessionsResponse(sessions=[Session(id='ff5c468d-813d-4299-b767-b1ef576103c1', app_name='local-trend-trawler', user_id='u_123', state={}, events=[], last_update_time=1757635751.549083)])

In [ ]:
session = await app.async_get_session(user_id="u_123", session_id=session.id)
session

Session(id='ff5c468d-813d-4299-b767-b1ef576103c1', app_name='local-trend-trawler', user_id='u_123', state={}, events=[], last_update_time=1757635751.549083)

The `async_stream_query` method returns a stream of events that represent the agent's execution trace.

In [ ]:
# for event in app.stream_query(
#     user_id = 'u_123',
#     session_id = session.id,
#     message = 'start'
# ):
#     print(event)

events = []
async for event in app.async_stream_query(
    user_id="u_123",
    session_id=session.id,
    message="start",
):
    events.append(event)

# The full event stream shows the agent's thought process
print("--- Full Event Stream ---")
for event in events:
    print(event)

# For quick tests, you can extract just the final text response
final_text_responses = [
    e for e in events
    if e.get("content", {}).get("parts", [{}])[0].get("text")
    and not e.get("content", {}).get("parts", [{}])[0].get("function_call")
]
if final_text_responses:
    print("\n--- Final Response ---")
    print(final_text_responses[0]["content"]["parts"][0]["text"])

INFO:root:gcs_folder: 2025_09_12_00_24
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:httpx:HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/hybrid-vertex/locations/us-central1/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:httpx:HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/hybrid-vertex/locations/us-central1/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
INFO:google_adk.google.adk.models.google_llm:Response 

--- Full Event Stream ---
{'invocation_id': 'e-e902191c-1f6e-4791-8855-0c582b1e83fe', 'author': 'root_agent', 'actions': {'state_delta': {'_state_init': True, 'gcs_folder': '2025_09_12_00_24', 'brand': 'Paul Reed Smith (PRS)', 'target_product': 'PRS SE CE24 Electric Guitar', 'target_audience': 'millennials who follow jam bands (e.g., Widespread Panic and Phish), respond positively to nostalgic messages', 'key_selling_points': 'The 85/15 S Humbucker pickups deliver a wide tonal range, from thick humbucker tones to clear single-coil sounds, making the guitar suitable for various genres.', 'target_search_trends': {'target_search_trends': []}}, 'artifact_delta': {}, 'requested_auth_configs': {}, 'requested_tool_confirmations': {}}, 'id': 'f4950370-e7a2-4c38-9165-db911c4057ea', 'timestamp': 1757636686.66057}
{'content': {'parts': [{'thought_signature': 'Cv0BAR_MhbZbko_f9oKU7oHu9bv68eYfX0ysTlSbCitd8Sp7ohaIZgq-iG2tpjqPkalPG6_86d8K8ACF7K--AIVQNK0uXPBTQTqIRWdqX0p2q4nRPMdC8zGZ6z-w_JkN4WIH2l2gQDi

In [21]:
event

{'content': {'parts': [{'thought_signature': 'Cs0CAR_MhbYYEzctd9mGT5z2O8bmWaxEZJrhEzNmy0QbtJr_2ZmyafDPzan6UdzLFOTcn3KnUir3kog2keextZqJgeJUIqql9r1UMwD09568lhFW2IX8h5Md7ZbsvUE-WMbv949xt6cWh5xP4gUxvrb5YXfwesb58izKjmXF6dbSzrO2kxbxkH8c7p9fjT9v6sUxGKw7NbpUKgYhsJSp98ub9CAF3mAkDIGqvib5MheX5HZkc7ckBhuz0AWnypFZ6jHeatPhmtQvvs7Fbe1HTXWdYRJ33bsp5EUipcUqbeOcCQ0L02-W1cEPH84_hE3XkruOl_YT3cfCEm35PUpgQD60jQ9ky6f2pREGtzageqsiT9cGZS3QYKkOWQ-QqIQhxGHkK2GqmW_fIaZYprYEB57vUDCH7dyRA5XcFJRJnchZ9sODToO2MDxju1RCvUBj',
    'text': 'To identify the most culturally relevant trends for PRS SE CE24 Electric Guitar, targeting millennials who follow jam bands and respond positively to nostalgic messages, I have selected the following trends from the provided list: "Lossless Spotify," "Polly Holliday," and "DNA." These trends offer unique avenues to connect with the target audience and highlight the product\'s key selling points, specifically its versatile tonal range and ability to evoke diverse musical experiences.\n\

In [ ]:
for event in await app.async_get_session(user_id = 'u_123', session_id = session.id):
    print(event)

('id', 'ff5c468d-813d-4299-b767-b1ef576103c1')
('app_name', 'local-trend-trawler')
('user_id', 'u_123')
('state', {'_state_init': True, 'gcs_folder': '2025_09_12_00_24', 'brand': 'Paul Reed Smith (PRS)', 'target_product': 'PRS SE CE24 Electric Guitar', 'target_audience': 'millennials who follow jam bands (e.g., Widespread Panic and Phish), respond positively to nostalgic messages', 'key_selling_points': 'The 85/15 S Humbucker pickups deliver a wide tonal range, from thick humbucker tones to clear single-coil sounds, making the guitar suitable for various genres.', 'target_search_trends': {'target_search_trends': ['Lossless Spotify', 'Polly Holliday', 'DNA']}, 'timer_start': 1757636686.6633418, 'request_count': 13, 'raw_gtrends': ['charlie kirk', 'dna', 'evergreen high school', 'adn', 'nato article 4', 'polly holliday', 'lossless spotify', 'is utah a red state', 'taylor frankie paul bachelorette', 'newsmax', 'ben shapiro', 'trump twitter', 'melissa hortman', 'richest man in the world', 

In [ ]:
await app.async_list_sessions(user_id="u_123")

ListSessionsResponse(sessions=[Session(id='ff5c468d-813d-4299-b767-b1ef576103c1', app_name='local-trend-trawler', user_id='u_123', state={'_state_init': True, 'gcs_folder': '2025_09_12_00_24', 'brand': 'Paul Reed Smith (PRS)', 'target_product': 'PRS SE CE24 Electric Guitar', 'target_audience': 'millennials who follow jam bands (e.g., Widespread Panic and Phish), respond positively to nostalgic messages', 'key_selling_points': 'The 85/15 S Humbucker pickups deliver a wide tonal range, from thick humbucker tones to clear single-coil sounds, making the guitar suitable for various genres.', 'target_search_trends': {'target_search_trends': ['Lossless Spotify', 'Polly Holliday', 'DNA']}, 'timer_start': 1757636686.6633418, 'request_count': 13, 'raw_gtrends': ['charlie kirk', 'dna', 'evergreen high school', 'adn', 'nato article 4', 'polly holliday', 'lossless spotify', 'is utah a red state', 'taylor frankie paul bachelorette', 'newsmax', 'ben shapiro', 'trump twitter', 'melissa hortman', 'rich

In [ ]:
await app.async_delete_session(user_id="u_123", session_id=session.id)

In [ ]:
await app.async_list_sessions(user_id="u_123")

ListSessionsResponse(sessions=[])

## Deploy Agent to Vertex AI Agent Engine
Once you are satisfied with your agent's local behavior, deploy it

### Create A Deployment

* [src code](https://github.com/googleapis/python-aiplatform/blob/main/vertexai/_genai/agent_engines.py#L1085) for agent `config`
* Note: after upgrading to `google-cloud-aiplatform==1.112.0` | `vertexai==1.112.0` see [agent engine migration](https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/agent-engine-migration#before) docs

In [28]:
VERSION="v1"
print(f"VERSION: {VERSION}")

root_agent.name, root_agent.description

VERSION: v1


('root_agent',
 'Determines culturally relevant Search trends to use for ad creatives.')

In [ ]:
remote_agent = client.agent_engines.create(
    agent=root_agent,
    config={
        "requirements": "./trend_trawler/requirements.txt",
        "extra_packages": ["./trend_trawler"],
        "staging_bucket": f"gs://{os.getenv('GOOGLE_CLOUD_STORAGE_BUCKET')}",
        "gcs_dir_name": f"adk-pipe/trend-trawler/{VERSION}/staging",
        "display_name": "trend-trawler",
        "description": root_agent.description,
        # "env_vars": ENV_VARS_DICT,
        # "service_account": SERVICE_ACCOUNT,
        "min_instances": 1,
        "max_instances": 100,
        "resource_limits": {"cpu": "4", "memory": "8Gi"},
        "container_concurrency": 9, # recommended value is 2 * cpu + 1
    },
)

INFO:vertexai_genai.agentengines:Deploying google.adk.agents.Agent as an application.
INFO:vertexai_genai.agentengines:Reading requirements from requirements='./trend_trawler/requirements.txt'
INFO:vertexai_genai.agentengines:Read the following lines: ['cloudpickle==3.1.1', 'db-dtypes==1.4.3', 'fastapi==0.116.1', 'google-adk==1.14.0', 'google-api-core==2.25.1', 'google-api-python-client==2.181.0', 'google-auth-httplib2==0.2.0', 'google-auth==2.40.3', 'google-cloud-aiplatform[agent_engines,adk]==1.112.0', 'google-cloud-bigquery==3.37.0', 'google-cloud-logging==3.12.1', 'google-cloud-storage==2.19.0', 'google-genai==1.36.0', 'numpy==2.3.3', 'pandas==2.3.2', 'tabulate==0.9.0', 'uvicorn==0.35.0', 'pillow==11.3.0', 'markdown==3.9']
INFO:vertexai_genai.agentengines:Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.112.0', 'pydantic': '2.11.7'}
INFO:vertexai_genai.agentengines:The following requirements are appended: {'pydantic==2.11.7'}
INFO:vertex

In [33]:
remote_agent

AgentEngine(api_resource.name='projects/934903580331/locations/us-central1/reasoningEngines/8085021260339216384')

uncomment to inspect the Agent Engine spec...

In [38]:
# remote_agent.api_resource

To use an existing Agent Engine deployment, use `remote_agent=client.agent_engines.get(name=remote_agent.api_resource.name)`

In [ ]:
# remote_agent=client.agent_engines.get(name=remote_agent.api_resource.name)

remote_agent.api_resource.name

'projects/934903580331/locations/us-central1/reasoningEngines/8085021260339216384'

### Test Deployment

> see [Use a Agent Development Kit agent](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/use/adk) for details re: session management and streaming query responses

To list all [supported operations](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/use/overview#supported-operations), run `remote_agent.operation_schemas()`

In [ ]:
# remote_agent.operation_schemas()

#### Manage sessions

> `AdkApp` uses cloud-based managed sessions after you deploy the agent to Vertex AI Agent Engine

In [ ]:
session = await remote_agent.async_create_session(user_id="u_123")
session

INFO:httpx:HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/934903580331/locations/us-central1/reasoningEngines/8085021260339216384:query "HTTP/1.1 200 OK"


{'events': [],
 'lastUpdateTime': 1757639253.985771,
 'userId': 'u_123',
 'id': '4333040634890813440',
 'state': {},
 'appName': '8085021260339216384'}

In [ ]:
await remote_agent.async_list_sessions(user_id="u_123")

INFO:httpx:HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/934903580331/locations/us-central1/reasoningEngines/8085021260339216384:query "HTTP/1.1 200 OK"


{'sessions': [{'events': [],
   'state': {},
   'id': '4333040634890813440',
   'userId': 'u_123',
   'appName': '8085021260339216384',
   'lastUpdateTime': 1757639253.985771}]}

In [ ]:
session = await remote_agent.async_get_session(user_id="u_123", session_id=session['id'])
session

INFO:httpx:HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/934903580331/locations/us-central1/reasoningEngines/8085021260339216384:query "HTTP/1.1 200 OK"


{'userId': 'u_123',
 'lastUpdateTime': 1757639253.985771,
 'state': {},
 'events': [],
 'id': '4333040634890813440',
 'appName': '8085021260339216384'}

In [ ]:
await remote_agent.async_delete_session(user_id="u_123", session_id=session['id'])

In [ ]:
await remote_agent.async_list_sessions(user_id="u_123")

INFO:httpx:HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/934903580331/locations/us-central1/reasoningEngines/8085021260339216384:query "HTTP/1.1 200 OK"


{'sessions': [{'appName': '8085021260339216384',
   'userId': 'u_123',
   'events': [],
   'state': {},
   'id': '4333040634890813440',
   'lastUpdateTime': 1757639253.985771}]}

#### Stream a response to a query

> *Note: the `session_id=` argument is optional. If it is not specified, a new session will be automatically created and used for serving that query.*

In [ ]:
async for event in remote_agent.async_stream_query(
    user_id="u_456",
    # session_id="SESSION_ID",  # Optional
    message="start",
):
  print(event)

#### Delete deployed Agent 

In [ ]:
delete_app = False

if delete_app and remote_agent:
    task = remote_agent.delete(force = True)

Delete Agent Engine backing LRO: projects/934903580331/locations/us-central1/operations/3574326089297690624


INFO:vertexai.agent_engines:Delete Agent Engine backing LRO: projects/934903580331/locations/us-central1/operations/3574326089297690624


Agent Engine deleted. Resource name: projects/934903580331/locations/us-central1/reasoningEngines/199218312813477888


INFO:vertexai.agent_engines:Agent Engine deleted. Resource name: projects/934903580331/locations/us-central1/reasoningEngines/199218312813477888


# Deploy `creative_agent`

In [7]:
# don't forget we have an `.env` file under `creative_agent` dir
!tree -I "trend_trawler|stash|imgs"

.
├── creative_agent
│   ├── agent.py
│   ├── callbacks.py
│   ├── config.py
│   ├── __init__.py
│   ├── prompts.py
│   ├── requirements.txt
│   ├── sub_agents
│   │   ├── campaign_researcher
│   │   │   ├── agent.py
│   │   │   └── __init__.py
│   │   ├── __init__.py
│   │   └── trend_researcher
│   │       ├── agent.py
│   │       └── __init__.py
│   └── tools.py
├── deploy-to-agent-engine.ipynb
├── poetry.lock
├── pyproject.toml
└── README.md

5 directories, 16 files


In [8]:
# Load environment variables set with agent (./creative_agent/.env)
dotenv.load_dotenv(dotenv_path = os.path.join('creative_agent', '.env'))

PROJECT_ID=os.environ.get("GOOGLE_CLOUD_PROJECT")
PROJECT_NUMBER=os.environ.get("GOOGLE_CLOUD_PROJECT_NUMBER")
GCS_BUCKET_NAME=os.environ.get("GOOGLE_CLOUD_STORAGE_BUCKET")
LOCATION=os.environ.get("GOOGLE_CLOUD_LOCATION")

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUMBER: {PROJECT_NUMBER}")
print(f"GCS_BUCKET_NAME: {GCS_BUCKET_NAME}")
print(f"LOCATION: {LOCATION}")

PROJECT_ID: hybrid-vertex
PROJECT_NUMBER: 934903580331
GCS_BUCKET_NAME: trend-trawler-deploy-ae
LOCATION: us-central1


In [9]:
# Vertex AI Client Initialization:
client = vertexai.Client(
    project=PROJECT_ID, 
    location=LOCATION,
) # pyright: ignore[reportCallIssue]

## Test locally

> Before deploying ADK agent to Vertex AI Agent Engine, test locally with the SDK.

In [11]:
from creative_agent.agent import root_agent

root_agent

LlmAgent(name='root_agent', description='Help with ad generation; brainstorm and refine ad copy and visual concept ideas with actor-critic workflows; generate final ad creatives.', parent_agent=None, sub_agents=[], before_agent_callback=<function _load_session_state at 0x7f6112d29080>, after_agent_callback=None, model='gemini-2.0-flash-001', instruction="**Role:** You are the orchestrator for a comprehensive ad content generation workflow.\n\n    **Objective:** Your goal is to generate a complete set of ad creatives including ad copy, images, and videos. To achieve this, use the **specialized tools and sub-agents** available to complete the **instructions** below.\n    \n    **You have access to specialized tools and sub-agents:**\n    1. Use the `combined_research_pipeline` tool to conduct web research on the campaign metadata and selected trends.\n    2. Use the `ad_creative_pipeline` tool to generate ad copies.\n    3. Use the `visual_generation_pipeline` tool to create visual conce

In [51]:
from vertexai.agent_engines import AdkApp

app = AdkApp(
    agent=root_agent,
    app_name="local-creative-agent",
    enable_tracing=True,
)

app

In [52]:
session = await app.async_create_session(user_id="u_123")
session

Session(id='eaf14365-fcff-42f8-9c53-60148933fb91', app_name='local-creative-agent', user_id='u_123', state={}, events=[], last_update_time=1757642208.224841)

In [53]:
await app.async_list_sessions(user_id="u_123")

ListSessionsResponse(sessions=[Session(id='eaf14365-fcff-42f8-9c53-60148933fb91', app_name='local-creative-agent', user_id='u_123', state={}, events=[], last_update_time=1757642208.224841)])

In [54]:
session = await app.async_get_session(user_id="u_123", session_id=session.id)
session

Session(id='eaf14365-fcff-42f8-9c53-60148933fb91', app_name='local-creative-agent', user_id='u_123', state={}, events=[], last_update_time=1757642208.224841)

invoke the `creative_agent` with a search trend:

> `"target_search_trend: 'taylor swift engaged'"`

*note: for local testing, can interupt agent execution (for time's sake)*

In [ ]:
events = []
async for event in app.async_stream_query(
    user_id="u_123",
    session_id=session.id,
    message="target_search_trend: 'taylor swift engaged'",
):
    events.append(event)

# The full event stream shows the agent's thought process
print("--- Full Event Stream ---")
for event in events:
    print(event)

# For quick tests, you can extract just the final text response
final_text_responses = [
    e for e in events
    if e.get("content", {}).get("parts", [{}])[0].get("text")
    and not e.get("content", {}).get("parts", [{}])[0].get("function_call")
]
if final_text_responses:
    print("\n--- Final Response ---")
    print(final_text_responses[0]["content"]["parts"][0]["text"])

In [56]:
event

{'content': {'parts': [{'function_call': {'id': 'adk-0c654595-ec56-4338-acd2-37e1cc1983b5',
     'args': {'request': 'Ad Idea 1: Eras of Sound. Headline: Your Next Era of Tone Starts Now. Body Text: Every artist creates eras, and yours deserves the best. From the classic jam sounds of the 90s and 00s to today\'s improvisational explorations, the PRS SE CE 24 provides unparalleled tonal freedom. Its 85/15 "S" pickups and coil-tapping capability let you seamlessly transition between powerful humbucker and bright single-coil sounds. Celebrate new beginnings with a guitar built for enduring artistry, just like the careers that shape generations. Call-to-Action: Craft Your Legacy: Get the PRS SE CE 24. Candidate Social Media Caption: What’s your next musical era? Ours involves the incredible PRS SE CE 24. Dial in any sound, any time. #PRS #SECE24 #MusicEra #JamSession #NewBeginnings #TaylorSwift; Ad Idea 2: Beyond the Headlines, Into the Humbuckers. Headline: So, Taylor’s Engaged… But Have 

In [57]:
for event in await app.async_get_session(user_id = 'u_123', session_id = session.id):
    print(event)

('id', 'eaf14365-fcff-42f8-9c53-60148933fb91')
('app_name', 'local-creative-agent')
('user_id', 'u_123')
('state', {'_state_init': True, 'gcs_folder': '2025_09_12_01_58', 'brand': 'Paul Reed Smith (PRS)', 'target_product': 'PRS SE CE24 Electric Guitar', 'target_audience': 'millennials who follow jam bands (e.g., Widespread Panic and Phish), respond positively to nostalgic messages', 'key_selling_points': 'The 85/15 S Humbucker pickups deliver a wide tonal range, from thick humbucker tones to clear single-coil sounds, making the guitar suitable for various genres.', 'target_search_trends': 'taylor swift engaged', 'img_artifact_keys': {'img_artifact_keys': []}, 'vid_artifact_keys': {'vid_artifact_keys': []}, 'final_select_ad_copies': {'final_select_ad_copies': []}, 'final_select_vis_concepts': {'final_select_vis_concepts': []}, 'timer_start': 1757642337.618894, 'request_count': 10, 'initial_gs_queries': "1.  Who is Taylor Swift dating?\n2.  What is the history of Taylor Swift's past rela

In [59]:
# await app.async_list_sessions(user_id="u_123")

await app.async_delete_session(user_id="u_123", session_id=session.id)

await app.async_list_sessions(user_id="u_123")

ListSessionsResponse(sessions=[])

## Deploy Agent to Vertex AI Agent Engine

### Create a deployment

In [12]:
VERSION="v2"
print(f"VERSION: {VERSION}")

root_agent.name, root_agent.description

VERSION: v2


('root_agent',
 'Help with ad generation; brainstorm and refine ad copy and visual concept ideas with actor-critic workflows; generate final ad creatives.')

In [13]:
remote_agent = client.agent_engines.create(
    agent=root_agent,
    config={
        "requirements": "./creative_agent/requirements.txt",
        "extra_packages": ["./creative_agent"],
        "staging_bucket": f"gs://{os.getenv('GOOGLE_CLOUD_STORAGE_BUCKET')}",
        "gcs_dir_name": f"adk-pipe/creative-agent/{VERSION}/staging",
        "display_name": "creative-trend-agent",
        "description": root_agent.description,
        # "env_vars": ENV_VARS_DICT,
        # "service_account": SERVICE_ACCOUNT,
        "min_instances": 1,
        "max_instances": 100,
        "resource_limits": {"cpu": "4", "memory": "8Gi"},
        "container_concurrency": 9, # recommended value is 2 * cpu + 1
    },
)

remote_agent

INFO:vertexai_genai.agentengines:Deploying google.adk.agents.Agent as an application.
INFO:vertexai_genai.agentengines:Reading requirements from requirements='./creative_agent/requirements.txt'
INFO:vertexai_genai.agentengines:Read the following lines: ['cloudpickle==3.1.1', 'db-dtypes==1.4.3', 'fastapi==0.116.1', 'google-adk==1.14.0', 'google-api-core==2.25.1', 'google-api-python-client==2.181.0', 'google-auth-httplib2==0.2.0', 'google-auth==2.40.3', 'google-cloud-aiplatform[agent_engines,adk]==1.112.0', 'google-cloud-bigquery==3.37.0', 'google-cloud-logging==3.12.1', 'google-cloud-storage==2.19.0', 'google-genai==1.36.0', 'numpy==2.3.3', 'pandas==2.3.2', 'tabulate==0.9.0', 'uvicorn==0.35.0', 'pillow==11.3.0', 'markdown==3.9']
INFO:vertexai_genai.agentengines:Identified the following requirements: {'google-cloud-aiplatform': '1.112.0', 'cloudpickle': '3.1.1', 'pydantic': '2.11.7'}
INFO:vertexai_genai.agentengines:The following requirements are appended: {'pydantic==2.11.7'}
INFO:verte

AgentEngine(api_resource.name='projects/934903580331/locations/us-central1/reasoningEngines/5117149105902059520')

In [14]:
# remote_agent=client.agent_engines.get(name=remote_agent.api_resource.name)

remote_agent.api_resource.name

'projects/934903580331/locations/us-central1/reasoningEngines/5117149105902059520'

### Test Deployment

#### manage sessions

In [15]:
session = await remote_agent.async_create_session(user_id="u_123")
session

INFO:httpx:HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/934903580331/locations/us-central1/reasoningEngines/5117149105902059520:query "HTTP/1.1 200 OK"


{'state': {},
 'userId': 'u_123',
 'appName': '5117149105902059520',
 'id': '2162587089474945024',
 'events': [],
 'lastUpdateTime': 1757644640.126548}

In [16]:
await remote_agent.async_list_sessions(user_id="u_123")

INFO:httpx:HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/934903580331/locations/us-central1/reasoningEngines/5117149105902059520:query "HTTP/1.1 200 OK"


{'sessions': [{'userId': 'u_123',
   'id': '2162587089474945024',
   'state': {},
   'events': [],
   'lastUpdateTime': 1757644640.126548,
   'appName': '5117149105902059520'}]}

In [17]:
await remote_agent.async_delete_session(user_id="u_123", session_id=session['id'])

await remote_agent.async_list_sessions(user_id="u_123")

INFO:httpx:HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/934903580331/locations/us-central1/reasoningEngines/5117149105902059520:query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/934903580331/locations/us-central1/reasoningEngines/5117149105902059520:query "HTTP/1.1 200 OK"


{'sessions': []}

#### stream query response

In [18]:
async for event in remote_agent.async_stream_query(
    user_id="u_456",
    # session_id="SESSION_ID",  # Optional
    message="target_search_trend: 'taylor swift engaged'",
):
  print(event)

INFO:httpx:HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/934903580331/locations/us-central1/reasoningEngines/5117149105902059520:streamQuery?alt=sse "HTTP/1.1 200 OK"


{'invocation_id': 'e-06a73d83-ea96-4589-a2fe-6dd3bb69e3fc', 'author': 'root_agent', 'actions': {'state_delta': {'_state_init': True, 'gcs_folder': '2025_09_12_02_37', 'brand': None, 'target_product': None, 'target_audience': None, 'key_selling_points': None, 'target_search_trends': '', 'img_artifact_keys': {'img_artifact_keys': []}, 'vid_artifact_keys': {'vid_artifact_keys': []}, 'final_select_ad_copies': {'final_select_ad_copies': []}, 'final_select_vis_concepts': {'final_select_vis_concepts': []}}, 'artifact_delta': {}, 'requested_auth_configs': {}, 'requested_tool_confirmations': {}}, 'id': 'e94e88ba-1e5a-4d14-bdad-fb07ffd1ddc7', 'timestamp': 1757644657.856998}
{'content': {'parts': [{'text': "Okay, let's start by memorizing the provided search trend.\n\n"}, {'function_call': {'id': 'adk-18e10644-10cb-4ea3-87e2-45cb366c407d', 'args': {'value': 'taylor swift engaged', 'key': 'target_search_trends'}, 'name': 'memorize'}}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {

In [ ]:
for event in await remote_agent.async_get_session(user_id = 'u_456', session_id = "e-06a73d83-ea96-4589-a2fe-6dd3bb69e3fc"):
    print(event)

#### delete deployed agent

In [ ]:
delete_app = False

if delete_app and remote_agent:
    task = remote_agent.delete(force = True)